Classifying Images of Clothing using TensorFlow

tf.keras




In [ ]:
!pip install -U tensorflow_datasets

In [ ]:
# Install and import dependencies
import tensorflow as tf

# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Use a subset of the Fashio MNIST dataset with 60k images for training and 10k images for testing
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# store class names here to use later when plotting the images:
class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

# Explore the data
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))


# Preprocess the data
# The value of each pixel in the image data is an integer in the range [0,255].
# For the model to work properly, these values need to be normalized to the range [0,1].
# Apply normalize function to each image in the test and train datasets.
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()


# Explore the preprocessed data:
# Take a single image, and remove the color dimension by reshaping
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))
# Plot the image - voila a piece of fashion clothing
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

# Display the first 25 images from the training set and display the class name below each image.
# Verify that the data is in the correct format and we're ready to build and train the network.
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()


# Setup the layers
# The basic building block of a neural network is the layer.
# A layer extracts a representation from the data fed into it.
# Hopefully, a series of connected layers results in a representation that is meaningful for the problem at hand.
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
    tf.keras.layers.Dense(128, activation = tf.nn.relu),
    tf.keras.layers.Dense( 10, activation = tf.nn.softmax)
])


# Compile the model:
model.compile(optimizer = 'adam',
              loss      = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics   = ['accuracy'])




## <b>This network has three layers:</b>
<p><b>input: </b><code>tf.keras.layers.Flatten</code></p>
This layer transforms the images from a 2d-array of 28 × 28 pixels, to a 1d-array of 784 pixels (28*28).
Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn, as it only reformats the data.

<p><b>hidden: </b><code>tf.keras.layers.Dense</code></p>
A densely connected layer of 128 neurons. Each neuron (or node) takes input from all 784 nodes in the previous layer, weighting that input according to hidden parameters which will be learned during training, and outputs a single value to the next layer.

<p><b>output: </b><code>tf.keras.layers.Dense</code></p>
A 128-neuron, followed by 10-node softmax layer. Each node represents a class of clothing. As in the previous layer, the final layer takes input from the 128 nodes in the layer before it, and outputs a value in the range [0, 1], representing the probability that the image belongs to that class. The sum of all 10 node values is 1.

<p><b>Note:</b></p>
Using softmax activation and SparseCategoricalCrossentropy() has issues and which are patched by the tf.keras model.
A safer approach, in general, is to use a linear output (no activation function) with SparseCategoricalCrossentropy(from_logits=True).

<p><b>Training the model</b></p>
First, we define the iteration behavior for the train dataset:

Repeat forever by specifying dataset.repeat() (the epochs parameter described below limits how long we perform training).
The dataset.shuffle(60000) randomizes the order so our model cannot learn anything from the order of the examples.
And dataset.batch(32) tells model.fit to use batches of 32 images and labels when updating the model variables.

Training is performed by calling the model.fit method:

Feed the training data to the model using train_dataset.
The model learns to associate images and labels.
The epochs=5 parameter limits training to 5 full iterations of the training dataset, so a total of 5 * 60000 = 300000 examples.
